# 7. 딥러닝을 위한 고급 도구

## 7-1 Sequential 모델을 넘어서: 케라스의 함수형 API

https://arxiv.org/abs/1512.03385

### 7-1-1 함수형 API 소개

In [ ]:
from keras import Input , layers

input_tensor = Input(shape=(32,))
dense = layers.Dense(32 , activation = 'relu')

output_tensor = dense(input_tensor)

### 7-1-2 다중 입력 모델

In [ ]:
from keras.models import Model
from keras import layers
from keras import Input

test_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape = (None,) , dtype='int32' , name='text')

embedded_text =  layers.Embedding( test_vocabulary_size , 64)(text_input)

encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,), dtype='int32' , name='question')

embedded_question = layers.Embedding( question_vocabulary_size , 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text , encoded_question], axis = -1)

answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input , question_input] , answer)
model.compile(optimizer = 'rmsprop' , loss = 'categorical_crossentropy', metrics = ['acc'])

In [ ]:
import numpy as np
from tensorflow.keras.utils import to_categorical

num_samples = 1000
max_length = 100

text = np.random.randint(1 , test_vocabulary_size, size = (num_samples , max_length))
question = np.random.randint(1, question_vocabulary_size, size = (num_samples , max_length))
answers = np.random.randint(0, answer_vocabulary_size , size = num_samples)
answers = to_categorical(answers)

model.fit([text , question] , answers, epochs = 10, batch_size = 128)

model.fit({'text':text , 'question':question} , answers, epochs=10 , batch_size = 128)

Epoch 1/10
8/8 [==============================] - 1s 122ms/step - loss: 6.2938 - acc: 0.0040
Epoch 2/10
8/8 [==============================] - 1s 118ms/step - loss: 6.1601 - acc: 0.0020
Epoch 3/10
8/8 [==============================] - 1s 111ms/step - loss: 6.0842 - acc: 0.0060
Epoch 4/10
8/8 [==============================] - 1s 104ms/step - loss: 6.0175 - acc: 0.0080
Epoch 5/10
8/8 [==============================] - 1s 108ms/step - loss: 5.9546 - acc: 0.0150
Epoch 6/10
8/8 [==============================] - 1s 119ms/step - loss: 5.8684 - acc: 0.0170
Epoch 7/10
8/8 [==============================] - 1s 107ms/step - loss: 5.7758 - acc: 0.0200
Epoch 8/10
8/8 [==============================] - 1s 105ms/step - loss: 5.6773 - acc: 0.0270
Epoch 9/10
8/8 [==============================] - 1s 107ms/step - loss: 5.5881 - acc: 0.0300
Epoch 10/10
8/8 [==============================] - 1s 109ms/step - loss: 5.4825 - acc: 0.0370
Epoch 1/10
8/8 [==============================] - 3s 119ms/step - los

### 7-1-3 다중 출력 모델

In [ ]:
from keras import layers
from keras import Input
from keras.models import Model
vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,) , dtype='int32' , name='posts')
embedded_posts = layers.Embedding(vocabulary_size , 256)(posts_input)
x = layers.Conv1D(128 , 5 , activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256 , 5 , activation ='relu')(x)
x = layers.Conv1D(256 , 5 , activation ='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256 , 5 , activation = 'relu')(x)
x = layers.Conv1D(256 , 5 , activation = 'relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128 , activation = 'relu')(x)

age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups , activation = 'softmax' , name = 'income')(x)
gender_prediction = layers.Dense(1 , activation='sigmoid' , name='gender')(x)

model = Model(posts_input , [age_prediction , income_prediction , gender_prediction])

In [ ]:
model.compile(optimizer = 'rmsprop' , loss = ['mse' , 'categorical_crossentropy' , 'binary_crossentropy'])

model.compile(optimizer = 'rmsprop' , loss = {'age' : 'mse' , 'income' : 'categorical_crossentropy' , 'gender' : 'binary_crossentropy' })

In [ ]:
model.compile(optimizer = 'rmsprop' , loss = ['mse' , 'categorical_crossentropy' , 'binary_crossentropy'] , loss_weights = [0.25 , 1. , 10.])

model.compile(optimizer = 'rmsprop' , loss = {'age':'mse' , 'income':'categorical_crossentropy' , 'gender' : 'binzry_crossentropy'} , loss_weights = {'age':0.25 , 'income':1. , 'gender': 10.})

### 7-1-4 층으로 구성된 비순환 유향 그래프

https://arxiv.org/abs/1409.4842.

https://arxiv.org/abs/1312.4400.

#### 인셉션 모듈

In [ ]:
from keras import layers
branch_a = layers.Conv2D(128 , 1 , activation = 'relu' , strides = 2)(x)

branch_b = layers.ConV2D(128 , 1 , activation = 'relu')(x)
branch_b = layers.Conv2D(128 , 1 , activation = 'relu' , strides=2)(branch_b)

branch_c = layers.AveragePooling2D(3 , strides = 2)(x)
branch_c = layers.Conv2D(128 , 3 , activation = 'relu' , strides = 2)(branch_c)

branch_d = layers.Conv2D(128 , 1 , activation='relu')(x)
branch_d = layers.Conv2D(128 , 3 , activation='relu')(branch_d)
branch_d = layers.Conv2D(128 , 3 , activation='relu' , strides=2)(branch_d)

output = layers.concatenate([branch_a , branch_b , branch_c , branch_d] , axis = -1)

#### 잔차 연결

### 7-1-5 층 가중치 공유

In [2]:
from keras import layers
from keras import Input
from keras.models import Model

lstm = layers.LSTM(32)
left_input = Input(shape=(None , 128))
left_output = lstm(left_input)

right_input = Input(shape=(None , 128))
right_output = lstm(right_input)

merged = layers.concatenate([left_output , right_output] , axis = -1)
predictions = layers.Dense(1 , activation='sigmoid')(merged)

model = Model( [left_input , right_input] , predictions )

### 7-1-6 층과 모델

In [4]:
from keras import layers
from tensorflow.keras import applications
from keras import Input

xception_base = applications.Xception(weights = None , include_top = False)
left_input = Input(shape=(250 , 250 , 3))
right_input = Input(shape=(250 , 250 ,3))

left_features = xception_base(left_input)
right_features = xception_base(right_input)

merged_features = layers.concatenate( [left_features , right_features] , axis = -1 )

### 7-1-7 정리

## 7-2 케라스 콜백과 텐서보드를 사용한 딥러닝 모델 검사와 모니터링

### 7-2-1 콜백을 사용하여 모델의 훈련 과정 제어하기

#### ModelCheckpoint와 EarlyStopping 콜백

In [ ]:
import keras

callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor = 'val_acc',
        patience=1,
    ),
    keras.callbacks.ModelCheckpoint(
        filepath = 'my_model.h5',
        monitor='val_loss',
        save_bset_only = True,
    )
]

model.compile(optimizer='rmsprop',
              loss = 'binary_crossentropy',
              metrics = ['acc'])

model.fit(x, y,
          epochs = 10,
          batch_size = 32,
          callbacks=callbacks_list,
          validation_data = (x_val , y_val))

#### ReduceLROnPlateau 콜백

In [ ]:
callbck_list = [
                keras.callbacks.ReduceLROnPlateau(
                    factor = 0.1 , 
                    patience = 10
                )
]

#### 자신만의 콜백 만들기

### 7-2-2 텐서보드 소개: 텐서플로의 시각화 프레임 워크

In [8]:
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 2000
max_len = 500

(x_train , y_train) , (x_test , y_test) = imdb.load_data(num_words = max_features)
x_train = sequence.pad_sequences( x_train , maxlen = max_len )
x_test =  sequence.pad_sequences( x_test , maxlen=max_len )

model = keras.models.Sequential()
model.add(layers.Embedding(max_features , 128 , input_length = max_len , name = 'embed'))
model.add(layers.Conv1D(32 , 7 , activation = 'relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32 , 7 , activation = 'relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer='rmsprop', loss = 'binary_crossentropy' , metrics=['acc'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed (Embedding)           (None, 500, 128)          256000    
                                                                 
 conv1d_3 (Conv1D)           (None, 494, 32)           28704     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 98, 32)           0         
 1D)                                                             
                                                                 
 conv1d_4 (Conv1D)           (None, 92, 32)            7200      
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 1)                

In [9]:
! mkdir my_log_dir

mkdir: cannot create directory ‘my_log_dir’: File exists


In [10]:
callbacks = [
             keras.callbacks.TensorBoard(
                 log_dir = 'my_log_dir',
                 histogram_freq = 1,
                 embeddings_freq = 1,
             )
]

history = model.fit(x_train , y_train,
                    epochs=20,
                    batch_size = 128,
                    validation_split=0.2,
                    callbacks=callbacks)

Epoch 1/20
157/157 [==============================] - 75s 469ms/step - loss: 0.6008 - acc: 0.6809 - val_loss: 0.4226 - val_acc: 0.8404
Epoch 2/20
157/157 [==============================] - 72s 456ms/step - loss: 0.4295 - acc: 0.8562 - val_loss: 0.5054 - val_acc: 0.8354
Epoch 3/20
157/157 [==============================] - 73s 463ms/step - loss: 0.3756 - acc: 0.8784 - val_loss: 0.4355 - val_acc: 0.8668
Epoch 4/20
157/157 [==============================] - 72s 460ms/step - loss: 0.3215 - acc: 0.9073 - val_loss: 0.6877 - val_acc: 0.8436
Epoch 5/20
157/157 [==============================] - 76s 484ms/step - loss: 0.2878 - acc: 0.9236 - val_loss: 0.6428 - val_acc: 0.8622
Epoch 6/20
157/157 [==============================] - 75s 474ms/step - loss: 0.2593 - acc: 0.9397 - val_loss: 0.6326 - val_acc: 0.8700
Epoch 7/20
157/157 [==============================] - 74s 471ms/step - loss: 0.2101 - acc: 0.9575 - val_loss: 0.6869 - val_acc: 0.8702
Epoch 8/20
157/157 [==============================] - 7

In [11]:
!tensorboard --logdir=my_log_dir

2022-03-24 13:50:40.714050: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.8.0 at http://localhost:6006/ (Press CTRL+C to quit)
http://localhost:6006/
^C


### 7-2-3 정리

## 7-3 모델의 성능을 최대로 끌어올리기

### 7-3-1 고급 구조 패턴

#### 배치 정규화

#### 깊이별 분리 합성곱

In [12]:
from keras.models import Sequential, Model
from keras import layers

height = 64
width = 64
channels = 3
num_classes = 10

model = Sequential()
model.add(layers.SeparableConv2D(32 , 3,
                                 activation='relu',
                                 input_shape=(height , width , channels,)))
model.add(layers.SeparableConv2D(64 , 3 , activation = 'relu'))
model.add(layers.SeparableConv2D(128 , 3 , activation = 'relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64 , 3 , activation = 'relu'))
model.add(layers.SeparableConv2D(128 , 3 , activation = 'relu'))
model.add(layers.GlobalAveragePooling2D())

model.add(layers.Dense(32 , activation='relu'))
model.add(layers.Dense(num_classes , activation='softmax'))

model.compile(optimizer = 'rmsprop' , loss = 'categorical_crossentropy')

### 7-3-2 하이퍼파라미터 최적화

In [ ]:
#Hyperopt
# Random

### 7-3-3 모델 앙상블

### 7-3-4 정리